In [38]:
import torch

In [39]:
print(torch.__version__)

1.12.1


In [40]:
torch.cuda.is_available()

False

In [41]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cpu


In [42]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [43]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
#import matplotlib.pyplot as plt
import numpy as np

In [44]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4,padding=0)

        self.conv2 = nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1)
        
        self.conv3=nn.Conv2d(in_channels=96,out_channels=96,kernel_size=1)
        
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=2)

        self.conv4=nn.Conv2d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2)

        self.conv5=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1)

        self.conv6=nn.Conv2d(in_channels=256,out_channels=256,kernel_size=1)
        
        self.pool6 = nn.MaxPool2d(kernel_size=3,stride=2)

        self.conv7=nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)

        self.conv8=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1)

        self.conv9=nn.Conv2d(in_channels=384,out_channels=384,kernel_size=1)

        self.fc1 = nn.Linear(in_features=(384*2*2),out_features=4096)

        self.fc2 = nn.Linear(in_features=4096,out_features=4096)

        self.fc3 = nn.Linear(in_features=4096,out_features=CLASSES)

    def forward(self, x):
        x = F.relu(self.conv1(x))

        x = F.relu(self.conv2(x))

        x = self.pool3(F.relu(self.conv3(x)))
        x = F.dropout(x, 0.5)

        x = F.relu(self.conv4(x))

        x = F.relu(self.conv5(x))

        x = self.pool6(F.relu(self.conv6(x)))
        x = F.dropout(x, 0.5)

        x = F.relu(self.conv7(x))

        x = F.relu(self.conv8(x))

        x = F.relu(self.conv9(x))
        x = F.dropout(x, 0.5)

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)

        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)

        x = self.fc3(x)
        
        return x

In [45]:
transform_conf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [46]:
BATCH_SIZE=16
train_dataset=datasets.MNIST('Users/saedunu/Healthcare/Assignments/Assignment2/MNIST_Data/',train=True,download=True,transform=transform_conf,)
test_dataset=datasets.MNIST('Users/saedunu/Healthcare/Assignments/Assignment2/MNIST_Data/',train=False,download=True,transform=transform_conf)

In [47]:
num_classes = 10

unique_labels = np.unique(train_dataset.targets.numpy())
if len(unique_labels) != num_classes:
    raise ValueError("Number of unique class labels in the dataset does not match num_classes.")


In [48]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [49]:
model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [50]:
def train(model,device,train_loader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        #print(classes.shape)
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(img)
        #print(output.shape)
        #--------------------
        expected_shape = (img.shape[0], num_classes)
        if output.shape != expected_shape:
            raise ValueError(f"Output shape {output.shape} does not match expected shape {expected_shape}.")
        #------------------
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_loader.dataset),
            100.*batch_ids / len(train_loader),loss.item()))

In [51]:
def test(model, device, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_loader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            #print(y_hat.shape)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [52]:
if __name__=='__main__':
    seed=42
    EPOCHS=2
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train
Train Epoch: 1 [59984/60000 (100%)]	Loss: 0.012603

 Test set: Avarage loss: -7,Accuracy:9710/10000 (97%)

inside train
Train Epoch: 2 [59984/60000 (100%)]	Loss: 0.019366

 Test set: Avarage loss: -7,Accuracy:9792/10000 (98%)

